<a href="https://colab.research.google.com/github/bekku/deeplearning/blob/master/GPU%E8%A9%95%E4%BE%A1%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as f
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import time
import random
!pip install deap
from deap import base, creator, tools


from google.colab import drive #インポート
drive.mount('/content/gdrive') #GoogleDriveのマウント
def load_cifar10(batch=128):
    train_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                # (0.4914,0.4822,0.4465),(0.247,0.243,0.261)
                                [0.5, 0.5, 0.5],  # RGB 平均
                                [0.5, 0.5, 0.5]   # RGB 標準偏差
                                )
                         ])),
        batch_size=batch,
        shuffle=True
    )

    test_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=False,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                # (0.4914,0.4822,0.4465),(0.247,0.243,0.261)
                                [0.5, 0.5, 0.5],  # RGB 平均
                                [0.5, 0.5, 0.5]   # RGB 標準偏差
                             )
                         ])),
        batch_size=batch,
    )

    return {'train': train_loader, 'test': test_loader}

torch.cuda.is_available()

     |████████████████████████████████| 163kB 8.5MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


True

# **評価関数**

In [2]:
def speed_acc_normalization(spe_max,spe_min,spe,acc_max,acc_min,acc):
    if spe_min<=spe<=spe_max:
        speed_normalization = 0.5*((spe_max-spe+1) - spe_min) / ( (spe_max - spe_min) )
    else:
        speed_normalization=0
    if acc_min<=acc<=acc_max:
      accuracy_normalization = 0.5*(acc - acc_min) / ( (acc_max - acc_min) )
    else:
        accuracy_normalization=0
    print("--------------------------------------------------------------------------------")
    print("速度:{} ,精度:{}".format(spe,acc))
    print("速度評価値:{} ,精度評価値:{}".format(speed_normalization,accuracy_normalization))
    print("評価指標:{}".format(speed_normalization + accuracy_normalization))
    print("--------------------------------------------------------------------------------")

    return speed_normalization + accuracy_normalization

# **教師モデルVGG**

In [3]:
class VGG16(nn.Module):
    def __init__(self): # , num_classes):
        super(VGG16, self).__init__()
        num_classes=10

        self.block1_output = nn.Sequential (
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block2_output = nn.Sequential (
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block3_output = nn.Sequential (
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block4_output = nn.Sequential (
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block5_output = nn.Sequential (
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 512),  #512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 32 ),  #4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(32, num_classes),  #4096
        )

    def forward(self, x,T):
        x = self.block1_output(x)
        # print(x.size())
        x = self.block2_output(x)
        # print(x.size())
        x = self.block3_output(x)
        # print(x.size())
        x = self.block4_output(x)
        # print(x.size())
        x = self.block5_output(x)
        # print(x.size())
        x = x.view(x.size(0), -1)
        # print(x.size())
        x = self.classifier(x)
        x = x/T
        return x

# **教師モデルの出力**

In [15]:
!nvidia-smi
Tnet: VGG16 = VGG16()
loader = load_cifar10()
device = torch.device('cuda:0')
Tnet.to(device)

param = torch.load('/content/gdrive/My Drive/コード/ランダムモデルの蒸留学習/GPU評価/VGG16_weight_200ep.pth')
Tnet.load_state_dict(param)

Tnet_speed_list = []
Tnet_accuracy_list = []

Tnet_speed = 0
correct = 0
Tnet_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = Tnet(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    Tnet_speed_list.append(t2-t1)
    Tnet_accuracy_list.append(correct/10000)

Tnet_speed = sum(Tnet_speed_list)/len(Tnet_speed_list)
Tnet_accuracy = sum(Tnet_accuracy_list)/len(Tnet_accuracy_list)
print(Tnet_speed_list)
print(Tnet_accuracy_list)

print(Tnet_speed)
print(Tnet_accuracy)


Sun Jul 12 22:44:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |   1323MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **生徒モデル**

In [13]:
class CreateVGG_2(torch.nn.Module):
    def __init__(self, VGG_model_list):
        super(CreateVGG_2, self).__init__()
        self.count = 0

        PRE_FILTER = 3
        for i in range(len(VGG_model_list)):
            if VGG_model_list[i]!=0:
                FILTER = VGG_model_list[i]
                self._modules["fc"+str(i+1)] = nn.Sequential (
                nn.Conv2d(PRE_FILTER, FILTER, kernel_size=3, padding=1),
                nn.BatchNorm2d(FILTER),
                nn.ReLU(inplace=True),
                )
                PRE_FILTER=FILTER

        self.classifier = nn.Sequential(
            nn.Linear(PRE_FILTER, PRE_FILTER), 
            nn.ReLU(True),
            nn.Linear(PRE_FILTER, PRE_FILTER//2 ), 
            nn.ReLU(True),
            nn.Linear(PRE_FILTER//2, 10), 
        )
        self.pool = torch.nn.MaxPool2d(2, 2) 
        

    def forward(self, x, T):
        for i in range(1,14):
            if "fc"+str(i) in self._modules:
                x = f.relu(self._modules["fc"+str(i)](x))
                # print(x.size())
            if i==2 or i==4 or i==7 or i==10 or i==13:
                x = self.pool(x)
                # print(x.size())
        x = x.view(x.size(0), -1)
        # print(x.size())
        x = self.classifier(x)
        # print(x.size())
        x = x/T
        return x

# **最適モデル①0.5-0.5**
## [2, 2, 3, 2, 0, 0, 4, 0, 3, 5, 3, 0, 0]

In [16]:
!nvidia-smi
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [2, 2, 3, 2, 0, 0, 4, 0, 3, 5, 3, 0, 0]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cuda:0')
net.to(device)

param = torch.load('/content/gdrive/My Drive/コード/ランダムモデルの蒸留学習/GPU評価/VGG_最適モデル_soft_05-05_2.pth')
net.load_state_dict(param)
loader = load_cifar10()


saiteki_1_speed_list = []
saiteki_1_accuracy_list = []

saiteki_1_speed = 0
correct = 0
saiteki_1_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    saiteki_1_speed_list.append(t2-t1)
    saiteki_1_accuracy_list.append(correct/10000)

saiteki_1_speed = sum(saiteki_1_speed_list)/len(saiteki_1_speed_list)
saiteki_1_accuracy = sum(saiteki_1_accuracy_list)/len(saiteki_1_accuracy_list)
print(saiteki_1_speed_list)
print(saiteki_1_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,saiteki_1_speed,Tnet_accuracy,0,saiteki_1_accuracy))

Sun Jul 12 22:46:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |   1325MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **最適モデル①0.75-0.25**
## [3, 1, 1, 1, 4, 2, 0, 0, 0, 2, 3, 0, 3]

In [19]:
!nvidia-smi
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [3, 1, 1, 1, 4, 2, 0, 0, 0, 2, 3, 0, 3]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cuda:0')
net.to(device)

param = torch.load('/content/gdrive/My Drive/コード/ランダムモデルの蒸留学習/GPU評価/VGG_最適モデル_soft_075-025_2.pth')
net.load_state_dict(param)
loader = load_cifar10()


saiteki_2_speed_list = []
saiteki_2_accuracy_list = []

saiteki_2_speed = 0
correct = 0
saiteki_2_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    saiteki_2_speed_list.append(t2-t1)
    saiteki_2_accuracy_list.append(correct/10000)

saiteki_2_speed = sum(saiteki_2_speed_list)/len(saiteki_2_speed_list)
saiteki_2_accuracy = sum(saiteki_2_accuracy_list)/len(saiteki_2_accuracy_list)
print(saiteki_2_speed_list)
print(saiteki_2_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,saiteki_2_speed,Tnet_accuracy,0,saiteki_2_accuracy))

Sun Jul 12 22:48:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |   1325MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **最適モデル③0.25-0.75**
## [0, 2, 3, 5, 5, 5, 6, 4, 3, 0, 6, 1, 6]

In [34]:
!nvidia-smi
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [2, 2, 3, 2, 5, 2, 3, 5, 3, 0, 6, 1, 6]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cuda:0')
net.to(device)

param = torch.load('/content/gdrive/My Drive/コード/ランダムモデルの蒸留学習/GPU評価/VGG_最適モデル_soft_025-075_2.pth')
net.load_state_dict(param)
loader = load_cifar10()



saiteki_3_speed_list = []
saiteki_3_accuracy_list = []

saiteki_3_speed = 0
correct = 0
saiteki_3_accuracy = 0


for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    saiteki_3_speed_list.append(t2-t1)
    saiteki_3_accuracy_list.append(correct/10000)

saiteki_3_speed = sum(saiteki_3_speed_list)/len(saiteki_3_speed_list)
saiteki_3_accuracy = sum(saiteki_3_accuracy_list)/len(saiteki_3_accuracy_list)
print(saiteki_3_speed_list)
print(saiteki_3_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,saiteki_3_speed,Tnet_accuracy,0,saiteki_3_accuracy))

Sun Jul 12 23:50:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |   2533MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **フィルム数1/2モデル**
## [2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5 ,5, 5]

In [24]:
!nvidia-smi
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5 ,5, 5]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cuda:0')
net.to(device)

param = torch.load('/content/gdrive/My Drive/コード/ランダムモデルの蒸留学習/GPU評価/半フィルモデル.pth')
net.load_state_dict(param)
loader = load_cifar10()



han_speed_list = []
han_accuracy_list = []

han_speed = 0
correct = 0
han_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    han_speed_list.append(t2-t1)
    han_accuracy_list.append(correct/10000)

han_speed = sum(han_speed_list)/len(han_speed_list)
han_accuracy = sum(han_accuracy_list)/len(han_accuracy_list)
print(han_speed_list)
print(han_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,han_speed,Tnet_accuracy,0,han_accuracy))

Sun Jul 12 22:51:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   1325MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **ランダムモデル①**
## [5, 2, 2, 3, 4, 2, 6, 0, 6, 1, 3, 1, 1]

In [27]:
!nvidia-smi
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [5, 2, 2, 3, 4, 2, 6, 0, 6, 1, 3, 1, 1]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cuda:0')
net.to(device)

param = torch.load('/content/gdrive/My Drive/コード/ランダムモデルの蒸留学習/GPU評価/ランダムモデル①.pth')
net.load_state_dict(param)
loader = load_cifar10()


randam_1_speed_list = []
randam_1_accuracy_list = []

randam_1_speed = 0
correct = 0
randam_1_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    randam_1_speed_list.append(t2-t1)
    randam_1_accuracy_list.append(correct/10000)

randam_1_speed = sum(randam_1_speed_list)/len(randam_1_speed_list)
randam_1_accuracy = sum(randam_1_accuracy_list)/len(randam_1_accuracy_list)
print(randam_1_speed_list)
print(randam_1_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,randam_1_speed,Tnet_accuracy,0,randam_1_accuracy))

Sun Jul 12 22:56:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    33W /  70W |   1325MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **ランダムモデル②**
## [3, 2, 5, 1, 3, 1, 3, 6, 3, 2, 3, 5, 3]

In [29]:
!nvidia-smi
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [3, 2, 5, 1, 3, 1, 3, 6, 3, 2, 3, 5, 3]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cuda:0')
net.to(device)

param = torch.load('/content/gdrive/My Drive/コード/ランダムモデルの蒸留学習/GPU評価/ランダムモデル②.pth')
net.load_state_dict(param)
loader = load_cifar10()


randam_2_speed_list = []
randam_2_accuracy_list = []

randam_2_speed = 0
correct = 0
randam_2_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    randam_2_speed_list.append(t2-t1)
    randam_2_accuracy_list.append(correct/10000)

randam_2_speed = sum(randam_2_speed_list)/len(randam_2_speed_list)
randam_2_accuracy = sum(randam_2_accuracy_list)/len(randam_2_accuracy_list)
print(randam_2_speed_list)
print(randam_2_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,randam_2_speed,Tnet_accuracy,0,randam_2_accuracy))

Sun Jul 12 22:58:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    32W /  70W |   1949MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------